# Determining Starbucks Demographics

In this notebook I will use the data cleaning performed in the processing notebook to attempt to determine what the core demographics of starbucks memebership are in a effort to see how targeted marketing could be implemented efficently in future on consumers. 

This process will involve running an unsupervised learning method to cluster the data into groups based on information provided in their profiles including age, gender, income and length of membership. It will also take into account how much users normally spend at starbucks on a weekly/per transaction basis. This information has been included as it will be interesting to understand which marketing techinques have most impact on high value consumers.

First, I will perform some further processing of the data to prepare it for the model (standard scalars etc.). Then I will run a K-means ml model to cluster the data and determine what the optimimun number of clusters describes the demographics appropriately. After this I will describe some interesting findings from the results to gain data understanding before the final step of this projetct in Part 2 (Modelling the prices for each demographic and seeing how they have performed for each marketing campagin. 

### Imports

In [1]:
# import general functions
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

# import functions for modelling
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

### Functions

In [ ]:

def k_means_predictions(data, n_clusters=10):
    """
    this can be used to see how to split the demographics data up for the analysis
    """
    # Apply feature scaling to the demographics data.
    scaler = StandardScaler()
    scaler.fit(data)
    input_data = scaler.transform(data)
    
    # run k-means over different cluster counts to see the best number of groups to split our demographic data into
    ad = {}
    cluster_counts = [2,3,4,5,6,7,8,9,10,12,14,16,18,20,22]
    for x in cluster_counts:
        kmeans = KMeans(n_clusters=x, max_iter=1000).fit(input_demo_data)
        ad[x] = kmeans.inertia_
    plt.plot(list(ad.keys()), list(ad.values()))
    plt.xlabel("Number of cluster")
    plt.ylabel("Average Distance")
    plt.show()
    
    # create and run the model
    final_kmeans = KMeans(n_clusters=n_clusters, max_iter=1000).fit(input_demo_data)
    predict = final_kmeans.predict(input_demo_data)
       
    # add the pedictions for the clustering onto the original dataset
    data['cluster'] = predict
    
    return data

### Global Variables

### Run Cleaning Functions

### Demographics Modelling

In [ ]:
# remove unwanted columns
demographics_input = user_data.drop(columns=['total sales','number of sales'])

In [ ]:
# Apply feature scaling to the demographics data.
scaler = StandardScaler()
scaler.fit(demographics_input)
input_demo_data = scaler.transform(demographics_input)

In [ ]:
# run k-means over different cluster counts to see the best number of groups to split our demographic data into
ad = {}
cluster_counts = [2,3,4,5,6,7,8,9,10,12,14,16,18,20,22]
for x in cluster_counts:
    kmeans = KMeans(n_clusters=x, max_iter=1000).fit(input_demo_data)
    ad[x] = kmeans.inertia_
plt.plot(list(ad.keys()), list(ad.values()))
plt.xlabel("Number of cluster")
plt.ylabel("Average Distance")
plt.show()

In [ ]:
# create and run the model
final_kmeans = KMeans(n_clusters=8, max_iter=1000).fit(input_demo_data)
predict = final_kmeans.predict(input_demo_data)

# add the pedictions for the clustering onto the original dataset
demographics_output = demographics_input
demographics_output['cluster'] = predict

### Modelling Analysis

In [ ]:
demographics_output.groupby('cluster')['female'].count()

In [ ]:
demographics_output.groupby('cluster').mean()

In [ ]:
demographics_output.groupby('cluster')['age'].describe()